# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121871e+02     1.548131e+00
 * time: 0.09265303611755371
     1     1.093828e+01     8.421688e-01
 * time: 1.3608238697052002
     2    -1.135611e+01     9.716784e-01
 * time: 1.5212600231170654
     3    -3.389773e+01     7.937760e-01
 * time: 1.6733129024505615
     4    -4.745001e+01     6.517369e-01
 * time: 1.840608835220337
     5    -5.677437e+01     2.431076e-01
 * time: 1.994182825088501
     6    -5.966014e+01     1.770690e-01
 * time: 2.101749897003174
     7    -6.081028e+01     6.692830e-02
 * time: 2.2210049629211426
     8    -6.128290e+01     8.481763e-02
 * time: 2.333582878112793
     9    -6.158603e+01     4.596983e-02
 * time: 2.4634180068969727
    10    -6.182245e+01     3.395222e-02
 * time: 2.574906826019287
    11    -6.199700e+01     2.657421e-02
 * time: 2.687274932861328
    12    -6.207046e+01     2.148712e-02
 * time: 2.8103678226470947
    13    -6.213686e+01     1.454296e-02
 * time: 2.926368951797485

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557676
    AtomicNonlocal      14.8522650
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485366 
    Xc                  -19.3336818

    total               -62.261666460840
